<h1>Why the BIG WIRES Act?  An Exploratory Data Analysis</h1>
<h2>Eric Lagally</h2>
<h2>ericlagally@gmail.com</h2>
<h3>January 2024</h3>

The BIG WIRES Act (H.R. 5551, S. 2827) was introduced in late 2023.  It is a bill that proposes to streamline the permitting process for the development of large, high-voltage electrical transmission lines in the United States.  The introduction to the text of the Act reads:
> Congress finds that—

> (1) interregional electric transmission is foundational to a modern grid and a growing economy based on affordable and abundant energy;

> (2) in recent decades, the interregional electric grid infrastructure of North America, and of the United States in particular, has stagnated;

> (3) since 2014, the total capacity of planned or newly constructed large-scale interregional transmission in North America has amounted to less than 1⁄3 that of South America, 1⁄6 that of Europe, and 1⁄30 that of China;

> (4) such a shortage of interregional transmission threatens the reliability of the electric grid of the United States and its resiliency to extreme weather events and cyber and physical attacks while artificially inflating energy prices for the people of the United States;

> (5) the damages resulting from that interregional transmission shortfall have been borne across the United States, in part during extreme weather events, including the 2022 freeze affecting the majority of the United States, the 2020 and 2022 heat waves in California, the 2019 heat wave and 2021 freeze in Texas, the 2018 cyclone on the East Coast, and the 2014 polar vortex in the Northeast and Midwest, and those extreme weather events collectively cost the people of the United States hundreds of billions of dollars;

> (6) new interregional transmission, and transfer capability achieved by means of other technologies and grid-operational practices, represents a unique opportunity to fortify the critical energy infrastructure of the United States while lowering the cost of living for families;

> (7) studies of interregional transmission routinely find benefit-cost ratios as high as 2.5, in line with real-world experience;

> (8) by relieving grid congestion and promoting more efficient grid planning and operation, new interregional transmission presents an all-of-the-above opportunity for the full energy-generating portfolio of renewable, fossil, and nuclear resources of the United States; and

> (9) interregional transmission represents a 2-way value proposition, with secure careers for workers in energy communities and competitively sourced, low-cost, reliable energy for industrial, commercial, and residential energy customers nationwide.

Points #4 and #8 above caught my eye because they speak to an electrical grid that is potentially unreliable and that does not fully leverage the clean-energy capacity in the United States.  Specifically I wondered: 

1. What is the current state of high-voltage electrical transmission in the U.S.?
2. What are the regional and national needs for electrical transmission in the U.S.?
3. What other factors (i.e., placement of clean electricity generation facilties and carbon-equivalent emmissions from other electricity generation)  motivate the construction of new high-voltage transmission lines?

Using data from the U.S. Energy Information Administraton (EIA) and other sources, I sought to address these questions using figures, images, and modeling in Python, the full code for which is found in the cells below.  Good analysis is transparent and builds on previous learning.  For that reason, I encourage your careful review to determine whether my analysis is correct and, if so, to build on it to answer other more advanced questions you may have. 

In [ ]:
# import statements
import pandas as pd
import os
import glob
import seaborn as sns
import openpyxl
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import geopandas as gpd
import pyarrow.parquet
import shapely.geometry
import numpy as np

The first step of my approach is to import various data sets.  These are taken from the EIA website, mostly those pages relating to state-specific data.[1]   

In [ ]:
def read_data(keyword):
    file_list = glob.glob("../data/*.xlsx")
    dfs = []
    # for the energy tab in the state summary Excel sheets
    if keyword == "energy": 
        columns = ["Category","2022","2021","2020","2019","2018","2017","2016","2015","2014","2013","2012","2011","2010","2009","2008","2007",
                   "2006","2005","2004","2003","2002","2001","2000","1999","1998","1997","1996","1995","1994","1993","1992","1991","1990",
                   "Percent Share 2000","Percent Share 2010","Percent Share 2020","Percent Share 2022"]           
        for f in file_list:
            #grab state abbreviation
            state = f[-7:-5].upper()
            df = pd.read_excel(f, sheet_name=5,engine="openpyxl",names=columns,skiprows=3, skipfooter=1)
            df["State"] = state
            dfs.append(df)
    # for the emissions tab in the state summary Excel sheets        
    if keyword == "emissions":
        columns = ["Emission Type","2022","2021","2020","2019","2018","2017","2016","2015","2014","2013","2012","2011","2010","2009","2008","2007",
                   "2006","2005","2004","2003","2002","2001","2000","1999","1998","1997","1996","1995","1994","1993","1992","1991","1990"]
        for f in file_list:
            # grab state abbreviation
            state = f[-7:-5].upper()
            df = pd.read_excel(f, sheet_name=7,engine="openpyxl",names=columns,skiprows=2, skipfooter=1)
            df["State"] = state
            dfs.append(df)
    return dfs

In [ ]:
def read_exports():
    columns = ["State","Net Energy Exports (trillion BTU)"]
    df = pd.read_csv("../data/net_exports.csv", names=columns, skiprows=1)
    
    us_state_abbr = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC"
    }
    region = {
    "Alabama": "east",
    "Alaska": "alaska",
    "Arizona": "west",
    "Arkansas": "east",
    "California": "west",
    "Colorado": "west",
    "Connecticut": "east",
    "Delaware": "east",
    "Florida": "east",
    "Georgia": "east",
    "Hawaii": "hawaii",
    "Idaho": "west",
    "Illinois": "east",
    "Indiana": "east",
    "Iowa": "east",
    "Kansas": "east",
    "Kentucky": "east",
    "Louisiana": "east",
    "Maine": "east",
    "Maryland": "east",
    "Massachusetts": "east",
    "Michigan": "east",
    "Minnesota": "east",
    "Mississippi": "east",
    "Missouri": "east",
    "Montana": "west",
    "Nebraska": "east",
    "Nevada": "west",
    "New Hampshire": "east",
    "New Jersey": "east",
    "New Mexico": "west",
    "New York": "east",
    "North Carolina": "east",
    "North Dakota": "east",
    "Ohio": "east",
    "Oklahoma": "east",
    "Oregon": "west",
    "Pennsylvania": "east",
    "Rhode Island": "east",
    "South Carolina": "east",
    "South Dakota": "east",
    "Tennessee": "east",
    "Texas": "texas",
    "Utah": "west",
    "Vermont": "east",
    "Virginia": "east",
    "Washington": "west",
    "West Virginia": "east",
    "Wisconsin": "east",
    "Wyoming": "west",
    "District of Columbia": "east"}
    
    # convert BTU to MWh for consistency with other data
    df["Net Energy Exports (MWh)"] = df["Net Energy Exports (trillion BTU)"]*293071.08
    # Map a dictionary of interconnection regions to the dataframe
    df["Region"] = df["State"].map(region)
    # Map a dictionary of state abbreviations to the dataframe
    df["State"] = df["State"].map(us_state_abbr)
   
    #Convert all abbreviations to uppercase
    df["State"] = df["State"].apply(lambda x:x.upper())

    #Remove a period after an abbreviation
    df["State"] = df["State"].apply(lambda x:x.split('.')[0])

    return df

In [ ]:
def format_data(dfs, keyword):
    df_new = []
    for df in dfs:
        if keyword == "energy":
            # grab value of row matching the index label of interest - only all generation together
            idx = df.index[df["Category"] == "Total electric industry"].tolist()
            keepbelow = [i for i in range(idx[0])]
            df = df.drop(keepbelow)
            # keep only 2022 data
            df = df[["State","Category", "2022"]]
            keeprows = ["Total electric industry","Battery","Coal","Hydroelectric","Natural gas","Nuclear","Other","Other biomass","Other gas","Petroleum",
                        "Solar","Wood","Wind"]
            df = df[df["Category"].isin(keeprows)]
            df.reset_index(drop=True)
        if keyword == "emissions":   
            idx = df.index[df["Emission Type"] == "Total emission rate (lbs/MWh)"].tolist()
            keepbelow = [i for i in range(idx[0]+1)]
            df = df.drop(keepbelow)
            df = df[["State","Emission Type", "2022"]]
            df.reset_index(drop=True)
        df_new.append(df)
        
    return df_new

In [ ]:
df_exports = read_exports()
df_energy = read_data("energy")
df_emissions = read_data("emissions")
df_energy_clean = format_data(df_energy,"energy")
df_emission_clean = format_data(df_emissions, "emissions")

<h3>Question 1</h3>

Question #1 focuses on the current state of high-voltage electrical transmission lines in the U.S.  How many are there?  What are their capacities?  Once we know that information, we can analyze about how this compares to the current need, which is Question #2.  

I began by reviewing a primer on the organization of electricity transmission in the United States.[2]

To begin the analysis of existing electricity transmission infrastructure, I loaded a GeoJSON file of all U.S. electrical transmission lines downloaded from the ArcGIS Hub.[3]  I filtered the file to include only those high-voltage transmission lines >345 kV, which number 71,085.

In [ ]:
# import the geojson file (in parquet format) containing electricity transmission lines
transmission = gpd.read_parquet("../data/US_Electric_Power_Transmission_Lines.parquet")

In [ ]:
print("Data file consists of %s rows" %transmission.size)
# identify unique voltage categories
voltages = transmission["VOLT_CLASS"].unique()
print("Voltage categories are %s" %voltages)
# filter out low voltage lines, keeping only high-voltage lines >345 kV
hi_v_transmission = transmission[transmission["VOLT_CLASS"].isin(["345","500","735 AND ABOVE"])]
# drop any lines without voltage class listed
hi_v_transmission.dropna(subset=["VOLT_CLASS"])
print("Filtered data set with 345 kV AC and larger transmission lines contains %s rows" %hi_v_transmission.size)
hi_v_transmission.head()

Next I plot the high-voltage transmission lines to show high-level information:

In [ ]:
# create plot of energy transmission lines

def plot_transmission(df):
    lats_low, lats_mid, lats_hi = ([] for i in range (3))
    lons_low, lons_mid, lons_hi = ([] for i in range (3))
    names_low, names_mid, names_hi = ([] for i in range(3))
    fig = go.Figure()
    for feature, name, volt in zip(df.geometry, df.SUB_2, df.VOLT_CLASS):
        
        if isinstance(feature, shapely.geometry.linestring.LineString):
            linestrings = [feature]
        elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
            linestrings = feature.geoms
        else:
            continue
        # create latitude, longitude, and names arrays depending on voltage of the transmission line
        if (volt == "345"):
            for linestring in linestrings:
                x, y = linestring.xy
                lats_low = np.append(lats_low, y)
                lons_low = np.append(lons_low, x)
                names_low = np.append(names_low, [name]*len(y))
            lats_low = np.append(lats_low, None)
            lons_low = np.append(lons_low, None)
            names_low = np.append(names_low, None)
        elif (volt == "500"):
            for linestring in linestrings:
                x, y = linestring.xy
                lats_mid = np.append(lats_mid, y)
                lons_mid = np.append(lons_mid, x)
                names_mid = np.append(names_mid, [name]*len(y))
            lats_mid = np.append(lats_mid, None)
            lons_mid = np.append(lons_mid, None)
            names_mid = np.append(names_mid, None)
        else:
            for linestring in linestrings:
                x, y = linestring.xy
                lats_hi = np.append(lats_hi, y)
                lons_hi = np.append(lons_hi, x)
                names_hi = np.append(names_hi, [name]*len(y))
            lats_hi = np.append(lats_hi, None)
            lons_hi = np.append(lons_hi, None)
            names_hi = np.append(names_hi, None)      
    fig.add_trace(go.Scattermapbox(name="345 kV",mode="lines",line={"color":"blue","width":1},lat=lats_low,lon = lons_low,
                                  hovertext=names_low,hoverinfo="text"))
    fig.add_trace(go.Scattermapbox(name="500 kV",mode="lines",line={"color":"green","width":1},lat=lats_mid,lon = lons_mid,
                                   hovertext=names_mid,hoverinfo="text"))
    fig.add_trace(go.Scattermapbox(name=">=735 kV",mode="lines",line={"color":"red","width":1},lat=lats_hi,lon = lons_hi,
                                   hovertext=names_hi,hoverinfo="text"))
    #fig.update_geos(scope="usa")
    fig.update_layout(title_text = '2022 High-Voltage AC Transmission Lines', mapbox_style="open-street-map", mapbox_zoom=3,
                      mapbox_center={"lat":40, "lon":-98.5},margin={"r":0,"t":0,"l":0,"b":0}) 
    fig.show()
                      

In [ ]:
plot_transmission(hi_v_transmission)

As I expected, there are many high-voltage transmission lines connecting the states within the eastern and western halves of the US continent, which form the Eastern Interconnection and Western Interconnection regions. However, relatively few high-voltage transmission lines connect these two regions (connections to the Texas Interconnection Region fare better). This is understandable for two reasons. First, transmission lines are run at high voltages to minimize the voltage loss as much as possible over long distances, but the longer the distance the greater the loss.  Therefore one line connecting the western-most states with the eastern-most states is infeasible.  Second, previous attempts to connect the Interconnections in the second half of the 20th century were disconnected because they were not stable.  This was due to frequency fluctuations inherent to mismatches between power generation and power usage in AC systems (see [4]).  Current existing connections between the eastern and western halves of the country number seven DC ties totaling <b>1.32 GW</b> total. Existing high-voltage DC (HVDC) ties are shown below:

![DCTIES](us-hvdc-ties.jpg) 

<h3>Question 2</h3>

Given the current state of electrical transmission in the United States, I next investigated how this compares to the amount of electricity generated at a state level.  

In [ ]:
# create choropleth showing net energy exports
fig = px.choropleth(df_exports,locations="State", locationmode="USA-states", color_continuous_scale="RdBu", color="Net Energy Exports (trillion BTU)", scope="usa",
                   labels={"Net Energy Exports (trillion BTU)":""})
fig.update_layout(
    title_text = '2022 Net Electricity Exports by State (trillion BTU)', height = 600, width = 900)
fig.show()

As the figure shows, some states exported electricity in 2022 while adjacent states imported electricity (examples are Washington and Idaho, Alabama and Georgia, and Pennsylvania and Ohio).  These electricity exports and imports are supported by existing transmission lines to exchange electricity between adjacent states.  However, there are other states (like California) that import a tremendous amount of electricity that is not balanced by electricity exports from adjacent states.  This speaks to the need for even longer-distance transmission lines that connect distant states and/or DC ties that enhance connectivity between Interconnection regions.  I next investigated the net electricity balance between the three major interconnection regions:

In [ ]:
def aggregate_exports(df):
    df["Region_Sum"] = df.groupby("Region")["Net Energy Exports (trillion BTU)"].transform('sum')
    return df

In [ ]:
df_exports = aggregate_exports(df_exports)
fig = px.choropleth(df_exports,locations="State", locationmode="USA-states", color_continuous_scale="RdBu", color="Region_Sum", scope="usa",
                   labels={"Region_Sum":"Net Electricity Generation (trillion BTU)"})
fig.update_layout(
    title_text = '2022 Net Electricity by Interconnection Region', height=600, width=900)
fig.show()

The above choropleth demonstrates that each interconnection region has unique needs.  The Western Interconnection region, as a whole, is a net exporter of electricity.  However, the Eastern and Texas interconnection regions are net importers of electricity, and this is not compensated for by the exports from the Western Interconnection region.  To compare these data to the power capacity of existing HVDC ties, it was necesary to convert from BTU (a unit of energy) to GW (a unit of power).  Converting the BTU values to GW requires dividing the annual total by 8760 to yield BTU/hour, then coverting this to GW (1 BTU/hr = 0.29307107 W).  Doing so reveals that the Eastern Interconnection region requires a net import of <b>1.11 GW</b> while the Texas interconnection region requires a net import of <b>1.63 GW</b>.  Comparing to the DC tie capacity above of 1.32 GW, it is clear that these ties are near or at the limit of their capacity. There are slight inaccuracies here because the interconnection regions do not occur strictly along state lines.  A county-level analysis would provide more accurate outcomes.

To the point above that the net imports from certain interconnection regions are not fully compensated by other regions, we also see that the United States must be a net importer of electricity from Canada and Mexico. As shown in the Lawrence Livermore National Laboratory Sankey chart below, in 2022 the United States imported 0.14 quads (quadrillion BTU), or 1.4 x 10<sup>14</sup> BTU, of electricity.[5] While that is a small percentage of the total electricity used in the United States in that year, it is nonetheless a tremendous amount of energy.

![USELECTRICITY](US_Energy_2022.png) 

<h3>Question 3</h3>

Understanding the present condition of electricity generation at the state and regional levels and the limitations on transfer of this energy, I explored other factors that motivate how and where electricity transmission may be required.  In particular, as the United States transitions to a more clean-energy future we expect that electrical demands will increase compared to current state due to increased use of electric cars, furnaces, heat pumps, water heaters, and so on.  Further, the carbon-equivalent emissions profiles of the electricity generation become important to consider.  Generating more electricity to meet future demand with high carbon-emission sources will not effectively address the rate of climate change.  

I therefore explored the carbon-equivalent emissions of the electricity generation at the state and regional levels, and compared these to the sources of new carbon-neutral electricity generation in the form of solar electricity stations, which are quickly increasing in capacity. I began with the electricity generation data from the state profiles available on the EIA website and collated them:

In [ ]:
def all_states_generation(df_list):
    master_df = pd.DataFrame(columns = ["State","Total electric industry","Battery","Coal","Hydroelectric",
                                        "Natural gas","Nuclear","Other","Other biomass","Other gas","Petroleum","Solar","Wind","Wood"])
    for df in df_list:
        state = df.iloc[0]["State"]
        df = df.set_index("Category",drop=True)
        df = df.T
        df = df.reset_index()
        df = df.rename(columns = {"index":"State"})
        df = df.drop(0)
        df.at[1,"State"] = state
        cols = list(df.columns)
        if "Wind" not in cols:
            df["Wind"] = 0
        entry = df.iloc[[0]]
        master_df = pd.concat([master_df,entry])
    master_df = master_df.fillna(0)
    master_df = master_df.replace(".",0)
    master_df = master_df.reset_index(drop=True)
    return master_df

In [ ]:
allstate_energy = all_states_generation(df_energy_clean)
allstate_energy.head()

I grouped categories of electricity generation into "Emissive" and "Non-emissive" to get a sense of which states have the highest and lowest proportions of electricity generation from carbon-neutral sources.  Emissive categories were Coal, Natural gas, Other, Other biomass, Petroleum, and Wood.  Non-emissive categories were Hydroelectric, Nuclear, Solar, and Wind.  

In [ ]:
emissive = ["Coal","Natural gas","Other","Other biomass","Other gas","Petroleum","Wood"]
non_emissive = ["Hydroelectric","Nuclear","Solar","Wind"]
allstate_energy["Emissive Sum"] = allstate_energy[emissive].sum(axis=1)
allstate_energy["Non-Emissive Sum"] = allstate_energy[non_emissive].sum(axis=1)
allstate_energy["Emissive Percent"] = 100*(allstate_energy["Emissive Sum"] / allstate_energy["Total electric industry"])
allstate_energy["Non-Emissive Percent"] = 100*(allstate_energy["Non-Emissive Sum"] / allstate_energy["Total electric industry"])
allstate_energy.head()

Plotting the relative fractions of emissive generation methods by state yielded:

In [ ]:
fig = px.choropleth(allstate_energy,locations="State", locationmode="USA-states", color_continuous_scale=["lightgreen","black"], color="Emissive Percent", 
                    scope="usa", labels={"Emissive Percent":"Percent"})
fig.update_layout(
    title_text = '2022 Percent of Electricity Generation from Emissive Sources by State', height=600,width=900)
fig.show()

The top five and bottom five states with the most emissive electricity generation in 2022 were:

In [ ]:
allstate_energy = allstate_energy.sort_values(by=["Emissive Percent"])
print("Top 5 Least Emissive Electricity Generation:")
print(allstate_energy[['State', 'Emissive Percent']].head(5))
allstate_energy = allstate_energy.sort_values(by=["Emissive Percent"], ascending = False)
print("Top 5 Most Emissive Electricity Generation:")
print(allstate_energy[['State', 'Emissive Percent']].head(5))

As well as the percentage of electricity generating emissions in each state, the rate of emissions by type is important because not all emissions have the same effect on the environment. Estimates are available for sulfur dioxide (contributes to formation of acid rain), nitrogen oxide (a general term for various nitrogen-oxygen compounds of which NO<sub>2</sub> is a greenhouse gas), and carbon dioxide (a well-known greenhouse gas).  Methane emissions are not captured in the EIA data because the bulk of these emissions are categorized as oil and gas production rather than consumption for electricity generation. For more information on this topic, I recommend a recent study using satellite data to map methane emissions.[6]

In [ ]:
def all_states_emissions(df_list):
    master_df = pd.DataFrame()
    for df in df_list:
        df = df.rename(columns = {"2022":"Amount (lbs/MWh)"})
        master_df = pd.concat([master_df,df])
    master_df = master_df.fillna(0)
    master_df = master_df.replace(".",0)
    master_df = master_df.reset_index(drop=True)
    return master_df

In [ ]:
# collect all emissions data in one data frame
allstate_emissions = all_states_emissions(df_emission_clean)
allstate_sulfur = allstate_emissions[allstate_emissions["Emission Type"] == "Sulfur dioxide"].sort_values(by=["Amount (lbs/MWh)"], ascending=False
                                                                                                         ).reset_index(drop=True)
allstate_nitrogen = allstate_emissions[allstate_emissions["Emission Type"] == "Nitrogen oxide"].sort_values(by=["Amount (lbs/MWh)"], ascending=False
                                                                                                           ).reset_index(drop=True)
allstate_carbon = allstate_emissions[allstate_emissions["Emission Type"] == "Carbon dioxide"].sort_values(by=["Amount (lbs/MWh)"], ascending=False
                                                                                                         ).reset_index(drop=True)
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Sulfur Dioxide", "Nitrogen Oxide", "Carbon Dioxide"))

fig.add_trace(go.Bar(x=allstate_sulfur["State"], y=allstate_sulfur["Amount (lbs/MWh)"],),
              1, 1)
fig.add_trace(go.Bar(x=allstate_nitrogen["State"], y=allstate_nitrogen["Amount (lbs/MWh)"],),
              2, 1)
fig.add_trace(go.Bar(x=allstate_carbon["State"], y=allstate_carbon["Amount (lbs/MWh)"],),
              3, 1)
fig.update_layout(height=600, width=900,
                  title_text="2022 Electricity Generation Emission Amounts by Type and State (lbs/MWh)", showlegend=False)
fig.show()

These data illustrate a wide range of climate-impact footprints for electricity generation and export. Of note, four of five least-emissive states in their electricity generation are all in the Western Interconnection region, while all of the five most-emissive states are in the Eastern Interconnection region.  What would be ideal is to have the ability to strengthen transfer of non-emissive electricity between states both within and between interconnection regions.  Examples include between Washington, Oregon, and Utah as well as between Illinois and Wisconsin, Indiana, and Missouri. The above analysis also demonstrates those places where non-emissive electricity generation needs to increase.  

I next calculated the total emissions rate (sulfur dioxide, nitrogen oxides, and carbon dioxide) for those states that exported electricity in 2022:

In [ ]:
#allstate_emissions.head()
allstate_emsum = allstate_emissions.groupby(["State"]).sum(["Amount (lbs/MWh)"]).reset_index()
df_exports["Net Energy Exports (trillion BTU)"]= df_exports["Net Energy Exports (trillion BTU)"].astype(int)
df_exports_only = df_exports[df_exports["Net Energy Exports (trillion BTU)"] > 0]
#allstate_emsum.head()
df_exports_em = df_exports_only.merge(allstate_emsum, how="left")
df_exports_em["Total Rate"] = df_exports_em["Amount (lbs/MWh)"]/2000
fig = px.choropleth(df_exports_em,locations="State", locationmode="USA-states", color_continuous_scale=["lightgreen","black"], color="Total Rate", 
                    scope="usa", labels={"Total Rate":"Total Emissions (short tons/MWh)"})
fig.update_layout(
    title_text = '2022 Net Electricity Export Emissions by State', height=600,width=900)
fig.show()

The states that exported low-emissions electricity in 2022 primarily relied on hydroelectric, solar, and wind power to do so. Recent years have seen a rapid increase in the construction of solar photovoltaic and wind electricity generation in the United States, at both the utility and state levels. Public datasets of the major solar installations and wind farms are available from the USGS. To analyze the locations of these installations compared to the need for non-emissive electricity by state, I first loaded the datasets:

In [ ]:
solar_fac = gpd.read_file("../data/uspvdb_v1_0_20231108.geojson")
wind_fac = gpd.read_file("../data/uswtdb_v6_1_20231128.geojson")


In [ ]:
solar_fac = solar_fac.sort_values(by="p_cap_ac", ascending=False)
wind_fac = wind_fac.sort_values(by="p_cap", ascending=False)
wind_fac = wind_fac.fillna(0)
wind_fac.head()

Each data set contains the state in which the facility is located, enabling a visualization of the total wind and solar capacity by state as well as the individual facility locations and capacities:

In [ ]:
solar_cap_by_state = solar_fac.groupby(["p_state"], as_index=False)["p_cap_ac"].sum()
wind_cap_by_state = wind_fac.groupby(["t_state"], as_index=False)["p_cap"].sum()
#solar_cap_by_state.reset_index()
#wind_cap_by_state

In [ ]:
trace1 = {
  "type": "scattergeo", 
  "lat": solar_fac["ylat"], 
  "lon": solar_fac["xlong"], 
  "marker": dict(size=(solar_fac["p_cap_ac"]/10))
}
trace2 = {
  "type": "choropleth", 
  "z": solar_cap_by_state["p_cap_ac"], 
  "locations": solar_cap_by_state["p_state"],
  "colorscale": "solar", 
  "locationmode": "USA-states",
  "colorbar_title":"Total Generation (MW)"
}
data = [trace1, trace2]
layout = {
  "geo": {"scope": "usa"}
}
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title_text = "2022 Solar Electricity Generation Facilites and Total Capacity", height=600,width=900)
fig.show()

In [ ]:
wind_fac = wind_fac[wind_fac["t_cap"] > 2000]

In [ ]:
trace1 = {
  "type": "scattergeo", 
  "lat": wind_fac["ylat"], 
  "lon": wind_fac["xlong"], 
  "marker": dict(size=(wind_fac["p_cap"]/70),opacity=0.2)
}
trace2 = {
  "type": "choropleth", 
  "z": wind_cap_by_state["p_cap"], 
  "locations": wind_cap_by_state["t_state"],
  "colorscale": "aggrnyl", 
  "locationmode": "USA-states",
  "colorbar_title":"Total Generation (MW)"
}
data = [trace1, trace2]
layout = {
  "geo": {"scope": "usa"}
}
fig = go.Figure(data=data, layout=layout)
fig.update_layout(
    title_text = "2022 Wind Turbine Electricity Generation Facilities and Total Capacity", height=600,width=900)
fig.show()

<h2>Conclusions</h2>

This analysis illustrated several major findings:
<ol>
    <li>The existing network of high-voltage electrical transmission lines within the United States is primarily among nearby states.</li>
    <li>Interconnections between regions in the United States is achieved through high-voltage DC ties, which are at or nearly at capacity.</li>
    <li>There are some states that export electricity and others that import electricity but these states are not always adjacent.  Some states import a large fraction of their electricity, which is not fully supported by neighboring states.</li>
    <li>There is a wide variety of types of electricity generation in use and different states present widely-varying emissions profiles for the electricity they generate.</li>
    <li>Among the states that exported electricity in 2022, there is a widely varying profile of emissions resulting from that electricity generation.  The lowest-emissions electricity from exporting states is scattered throughout the country.</li>
    <li>The use of solar and wind for electricity generation is not evenly distributed across the states, and opportunties exist to enhance the transmission of electricity from those states producing it to adjacent states, especially those with high-emissions electricity generation and/or that are net importers of electricity.</li>
</ol>

These findings, when taken together, paint a picture of an existing need for increased electricity transmission capacity, especially to support future electricity demand due to clean energy efforts through other bills, like the Inflation Reduction Act.  Although I did not study it here, the rate of solar electricity generation is increasing, which is expected to increase this transmission need in the future. This analysis does not indicate whether the BIG WIRES Act is the optimal approach for such capacity improvement, but does support Points #4 and #8 in the introduction.

<h2>References</h2>

[1] U.S. Energy Information Administration, state search:  https://www.eia.gov/state/search/?sid=US

[2] https://www.ncsl.org/environment-and-natural-resources/electric-transmission-planning-a-primer-for-state-legislatures

[3] https://hub.arcgis.com/datasets/fedmaps::u-s-electric-power-transmission-lines/about

[4] https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=8594689, Section III

[5] https://flowcharts.llnl.gov/commodities/energy

[6] https://www.science.org/doi/10.1126/science.abj4351

